In [ ]:
import os
import h5py
import time
import math
import ctypes
import numpy as np
import pandas as pd
from multiprocessing.sharedctypes import RawArray

In [ ]:
from pathlib import Path
os.chdir(Path(os.getcwd()).parent)

In [ ]:
from libs.quotation import Quotation
from libs.tdx import TDX

In [ ]:
class DailyData:
    
    def __init__(self, symbols, check_points, buffers=None):
        
        if buffers is None:
            self.buffers = {}

            buffer_symbols = RawArray(ctypes.c_uint32, 6*len(symbols))
            self.buffers['symbols'] = buffer_symbols
            self.symbols = np.frombuffer(buffer_symbols, dtype='<U6')

            buffer_names = RawArray(ctypes.c_uint32, 4*len(symbols))
            self.buffers['names'] = buffer_names
            self.names = np.frombuffer(buffer_names, dtype='<U4')

            buffer_check_points = RawArray(ctypes.c_uint32, len(check_points))
            self.buffers['check_points'] = buffer_check_points
            self.check_points = np.frombuffer(buffer_check_points, dtype='<u4')

            basics_cols = ['zt_price', 'dt_price', 'ma5vpm', 'mcap', 'sum4', 'sum9', 'sum19', 'sum29', 'sum59']
            buffer_basics = RawArray(ctypes.c_float, len(symbols)*len(basics_cols))
            self.buffers['basics'] = buffer_basics
            arr = np.frombuffer(buffer_basics, dtype='<f4')
            self.basics = arr.reshape((len(symbols), len(basics_cols)))

            snapshots_cols = ['open','close','now','high','low','turnover','volume']
            buffer_snapshots = RawArray(ctypes.c_float, len(check_points)*len(symbols)*len(snapshots_cols))
            self.buffers['snapshots'] = buffer_snapshots
            arr = np.frombuffer(buffer_snapshots, dtype='<f4')
            self.snapshots = arr.reshape((len(check_points), len(symbols), len(snapshots_cols)))

            statistic_cols = ['zhangfu', 'junjia', 'liangbi', 'zhangsu', 'tingban', 'sum5', 'sum10', 'sum20', 'sum30', 'sum60']
            buffer_statistic = RawArray(ctypes.c_float, len(check_points)*len(symbols)*len(statistic_cols))
            self.buffers['statistic'] = buffer_statistic
            arr = np.frombuffer(buffer_statistic, dtype='<f4')
            self.statistic = arr.reshape((len(check_points), len(symbols), len(statistic_cols)))
            
        else:
            self.buffers = buffers
            # symbols
            self.symbols = np.frombuffer(buffers['symbols'], dtype='<U6')
            # names
            self.names = np.frombuffer(buffers['names'], dtype='<U4')
            # check_points, dtype='<u4'
            self.check_points = np.frombuffer(buffers['check_points'], dtype='<u4')
            # basics
            basics_cols = ['zt_price', 'dt_price', 'ma5vpm', 'mcap', 'sum4', 'sum9', 'sum19', 'sum29', 'sum59']
            self.basics = np.frombuffer(buffers['basics'], dtype='<f4')
            self.basics = self.basics.reshape((len(self.symbols), len(basics_cols)))
            # snapshots
            snapshots_cols = ['open','close','now','high','low','turnover','volume']
            self.snapshots = np.frombuffer(buffers['snapshots'], dtype='<f4')
            self.snapshots = self.snapshots.reshape((len(check_points), len(self.symbols), len(snapshots_cols)))
            # statistic
            statistic_cols = ['zhangfu', 'junjia', 'liangbi', 'zhangsu', 'tingban', 'sum5', 'sum10', 'sum20', 'sum30', 'sum60']
            self.statistic = np.frombuffer(buffers['statistic'], dtype='<f4')
            self.statistic = self.statistic.reshape((len(check_points), len(self.symbols), len(statistic_cols)))
        
    
    async def prepare(self, symbols, check_points):
        q = Quotation(symbols)
        snapshot = await q.snapshot()

        self.symbols[:] = symbols
        self.names[:] = [ snapshot[symbol]['name'] for symbol in symbols ]
        self.check_points[:] = check_points

        self.basics.fill(np.nan)
        self.snapshots.fill(np.nan)
        self.statistic.fill(np.nan)

        market_values = await q.get_market_values()

        await q.exit()
        
        tdx = TDX()
        assert tdx.is_local_tdx_data_outdated() is not True
        klines = tdx.kline(symbols)

        for _, symbol in enumerate(symbols):
            self.basics[_, 0] = market_values[symbol]['zt_price']
            self.basics[_, 1] = market_values[symbol]['dt_price']
            self.basics[_, 3] = market_values[symbol]['mcap']

            if symbol not in klines:
                continue
                
            kline = klines[symbol]
            self.basics[_, 2] = kline.iloc[0-min(5,len(kline)):]['volume'].sum()/1200
            self.basics[_, 4] = kline.iloc[-4:]['close'].sum() if len(kline) >= 4 else np.nan
            self.basics[_, 5] = kline.iloc[-9:]['close'].sum() if len(kline) >= 9 else np.nan
            self.basics[_, 6] = kline.iloc[-19:]['close'].sum() if len(kline) >= 19 else np.nan
            self.basics[_, 7] = kline.iloc[-29:]['close'].sum() if len(kline) >= 29 else np.nan
            self.basics[_, 8] = kline.iloc[-59:]['close'].sum() if len(kline) >= 59 else np.nan
            
        self.post_init()
            
    def post_init(self):
        self.check_interval = self.check_points[2] - self.check_points[1]
        start_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:15:00'
        start_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:30:00'
        start_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 13:00:00'
        end_time_0_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 09:25:00'
        end_time_1_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 11:30:00'
        end_time_2_str = f'{time.strftime("%Y-%m-%d", time.localtime(self.check_points[0]))} 15:00:00'
        start_time_0 = int(time.mktime(time.strptime(start_time_0_str, '%Y-%m-%d %H:%M:%S')))
        start_time_1 = int(time.mktime(time.strptime(start_time_1_str, '%Y-%m-%d %H:%M:%S')))
        start_time_2 = int(time.mktime(time.strptime(start_time_2_str, '%Y-%m-%d %H:%M:%S')))   
        end_time_0 = int(time.mktime(time.strptime(end_time_0_str, '%Y-%m-%d %H:%M:%S')))
        end_time_1 = int(time.mktime(time.strptime(end_time_1_str, '%Y-%m-%d %H:%M:%S')))
        end_time_2 = int(time.mktime(time.strptime(end_time_2_str, '%Y-%m-%d %H:%M:%S')))   
        
        start_time_idx_0 = np.where(self.check_points == start_time_0)[0][0]
        start_time_idx_1 = np.where(self.check_points == start_time_1)[0][0]
        start_time_idx_2 = np.where(self.check_points == start_time_2)[0][0]
        end_time_idx_0 = np.where(self.check_points == end_time_0)[0][0]
        end_time_idx_1 = np.where(self.check_points == end_time_1)[0][0]
        end_time_idx_2 = np.where(self.check_points == end_time_2)[0][0]
        self.active_time_blocks = [
            ((start_time_idx_0, start_time_0, start_time_0_str),(end_time_idx_0, end_time_0, end_time_0_str)),
            ((start_time_idx_1, start_time_1, start_time_1_str),(end_time_idx_1, end_time_1, end_time_1_str)),
            ((start_time_idx_2, start_time_2, start_time_2_str),(end_time_idx_2, end_time_2, end_time_2_str))
        ]
            
    
    def get_securities(self):
        return pd.DataFrame({
            "symbol": self.symbols,
            "name": self.names,
            "zt_price": self.basics[:,0],
            "dt_price": self.basics[:,1],
            "ma5vpm": self.basics[:,2],
            "mcap": self.basics[:,3],
            "sum4": self.basics[:,4],
            "sum9": self.basics[:,5],
            "sum19": self.basics[:,6],
            "sum29": self.basics[:,7],
            "sum59": self.basics[:,8]
        }).set_index('symbol')
    
    
    # check_point: timestamp or %H:%M:%S
    def get_snapshot(self, check_point):
        if type(check_point) == float or type(check_point) == int:
            index = np.where(self.check_points == check_point)[0][0]
        elif type(check_point) == str:
            index = [ time.strftime('%H:%M:%S', time.localtime(cp)) for cp in self.check_points ].index(check_point)
            
        return pd.DataFrame({
            "datetime": [ check_point if type(check_point) == str else time.strftime('%H:%M:%S', time.localtime(check_point)) for _ in range(len(self.symbols)) ], 
            "timestamp": [ self.check_points[index] if type(check_point) == str else check_point for _ in range(len(self.symbols)) ],
            "symbol": self.symbols,
            "name": self.names,
            "open": self.snapshots[index,:,0],
            "close": self.snapshots[index,:,1],
            "now": self.snapshots[index,:,2],
            "high": self.snapshots[index,:,3],
            "low": self.snapshots[index,:,4],
            "turnover": self.snapshots[index,:,5],
            "volume": self.snapshots[index,:,6],
            "zhangfu": self.statistic[index,:,0],
            "junjia": self.statistic[index,:,1],
            "liangbi": self.statistic[index,:,2],
            "zhangsu": self.statistic[index,:,3],
            "tingban": self.statistic[index,:,4],
            "fsto": self.statistic[index,:,5]
        })
    
    # dt: %Y%m%d
    def save(self, dt=None, gzip_level=4):
        date = time.strftime("%Y%m%d", time.localtime()) if dt is None else dt
        folder = os.path.join(os.getcwd(), 'storage')
        if not os.path.exists(folder):
            os.mkdir(folder)
        file = os.path.join(folder, f'{date}.hdf5')
        
        if os.path.exists(file):
            print('文件 [', file, '] 已经存在，将被删除 ... ... ', end='')
            os.remove(file)
            print('已被删除')
            
        symbols = np.char.encode(self.symbols, encoding='utf-8')
        names = np.char.encode(self.names, encoding='utf-8')
        with h5py.File(file, "a") as f:
            f.create_dataset(u"symbols", data=symbols, compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"names", data=names, compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"check_points", data=self.check_points, compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"basics", data=self.basics, compression="gzip", compression_opts=gzip_level)
            f.create_dataset(u"snapshots", data=self.snapshots, compression="gzip", compression_opts=gzip_level)
#             f.create_dataset(u"statistic", data=self.statistic, compression="gzip", compression_opts=gzip_level)
            
        print('行情 已经 写入文件：', file)
            
    # dt: %Y%m%d
    @staticmethod
    def load(dt=None):
        date = time.strftime("%Y%m%d", time.localtime()) if dt is None else dt
        file = os.path.join(os.getcwd(), 'storage', f'{date}.hdf5')
        assert os.path.exists(file), 'file['+file+'] does not exists!'
        
        with h5py.File(file, "a") as f:
            symbols = np.char.decode(f[u'symbols'], 'utf-8')
            check_points = f[u'check_points'][:]
            
            data = DailyData(symbols, check_points)
            data.symbols[:] = symbols
            data.names[:] = np.char.decode(f[u'names'], 'utf-8')
            data.check_points[:] = check_points
            data.basics[:,:] = f[u'basics'][:,:]
            data.snapshots[:,:,:] = f[u'snapshots'][:,:,:]
#             data.statistic[:,:,:] = f[u'statistic'][:,:,:]
            
        data.post_init()
        return data
    
    
    def get_ma5pm_anchor_idx(self, idx):
#         st = time.time()

        ck = self.check_points[idx]

        if ck <= self.active_time_blocks[0][0][1]+300:
            ma5pm_anchor_idx = self.active_time_blocks[0][0][0] - 1

        elif self.active_time_blocks[0][1][1] < ck < self.active_time_blocks[1][0][1]:
            ma5pm_anchor_idx = int(self.active_time_blocks[0][1][0] - 300/self.check_interval)

        elif self.active_time_blocks[1][0][1] <= ck <= self.active_time_blocks[1][0][1]+300:
            ma5pm_anchor_idx = self.active_time_blocks[1][0][0] - 1

        elif self.active_time_blocks[1][1][1] < ck < self.active_time_blocks[2][0][1]:
            ma5pm_anchor_idx = int(self.active_time_blocks[1][1][0] - 300/self.check_interval)

        elif self.active_time_blocks[2][0][1] <= ck <= self.active_time_blocks[2][0][1]+300:
            result = math.ceil(self.check_points[idx]/60)*60-300
            offset = int((result-self.check_points[idx])/self.check_interval)

            ma5pm_anchor_idx = int(max(idx+offset-2, self.active_time_blocks[1][1][0] - 240/self.check_interval))

            if ma5pm_anchor_idx == self.active_time_blocks[1][1][0]:
                ma5pm_anchor_idx += 1

        elif self.active_time_blocks[2][1][1] < ck:
            ma5pm_anchor_idx = int(self.active_time_blocks[2][1][0]-300/self.check_interval)

        else:
            result = math.ceil(self.check_points[idx]/60)*60-300
            offset = int((result-self.check_points[idx])/self.check_interval)

            ma5pm_anchor_idx = idx+offset

#         et = time.time()

#         print(
#             idx, ' : ', ma5pm_anchor_idx, '       ', 
#             time.strftime("%H:%M:%S", time.localtime(self.check_points[idx])),
#             ' ==> ',
#             time.strftime("%H:%M:%S", time.localtime(self.check_points[ma5pm_anchor_idx])), '       ',
#             ck, ' : ', self.check_points[ma5pm_anchor_idx], '       ',
#             et-st
#         )

        return ma5pm_anchor_idx


    def get_time_lapse(self, idx):
    
        ck = self.check_points[idx]
        offset = 0
        if ck < self.active_time_blocks[1][0][1]:
            start_time = self.active_time_blocks[0][0][1]
            if ck > self.active_time_blocks[0][1][1]:
                ck = self.active_time_blocks[0][1][1]
        elif ck < self.active_time_blocks[2][0][1]:
            start_time = self.active_time_blocks[1][0][1]
            if ck > self.active_time_blocks[1][1][1]:
                ck = self.active_time_blocks[1][1][1]
        else:
            start_time = self.active_time_blocks[2][0][1]
            if ck > self.active_time_blocks[2][1][1]:
                ck = self.active_time_blocks[2][1][1]
            offset = 120

        return max(int(math.ceil((ck - start_time)/60)),1)+offset

In [5]:
from libs.utils import Utils

In [6]:
Utils.update_symbols()

In [7]:
snapshot_interval = 5
check_points = Utils.get_check_points()
symbols = Utils.get_running_symbols()

In [8]:
data = DailyData(symbols, check_points)

In [ ]:
await data.prepare(symbols, check_points)

In [ ]:
data.save()

In [ ]:
data = DailyData.load()

In [ ]:
data.get_securities()

In [ ]:
data.start_points

In [ ]:
'300933' in data.symbols

In [ ]:
data.get_snapshot('09:16:00')

In [ ]:
data.get_snapshot(1611018960)

In [ ]:
data.statistic.shape

In [ ]:
########################   测试 get_ma5pm_anchor_idx, get_time_lapse  ####################################
for _, ck in enumerate(data.check_points):
    print(_, ' | ',
          time.strftime("%H:%M:%S", time.localtime(ck)),' -> ',
          time.strftime("%H:%M:%S", time.localtime(data.check_points[data.get_ma5pm_anchor_idx(_)])), ' | ',
          data.get_time_lapse(_)
         )

In [ ]:
idx = 2959

In [ ]:
%%timeit
data.get_ma5pm_anchor_idx(idx)

In [ ]:
%%timeit
data.get_time_lapse(idx)